# German Llama fine-tuned with mlx for diagnosis


In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Define the number of samples per label
samples_per_label = 17

In [18]:
# Load and preprocess the dataset
df = pd.read_csv('../../data/balanced_classification_dataset.csv')
df = df.rename(columns={'data': 'prompt', 'DIA_text': 'completion'})
df = df.drop(df.columns.difference(['prompt', 'completion', 'label']), axis=1)

# Switch the places of 'prompt' and 'completion' columns
columns = list(df.columns)
col1, col2 = columns.index('prompt'), columns.index('completion')
columns[col1], columns[col2] = columns[col2], columns[col1]
df = df[columns]

# Create the test DataFrame
samples_per_label = 17
test = pd.DataFrame()

for label in df['label'].unique():
    label_subset = df[df['label'] == label].sample(n=samples_per_label, random_state=42)
    test = pd.concat([test, label_subset])

# Remove 'label' column and reset index for the test DataFrame

test.reset_index(drop=True, inplace=True)
test = test.drop(["label"], axis=1)

# Create the train DataFrame by dropping test rows
train = df.drop(test.index)

# Split the train DataFrame into train and valid DataFrames with 80:20 ratio
train_df_list = []
valid_df_list = []

for label in train['label'].unique():
    label_df = train[train['label'] == label]
    train_label, valid_label = train_test_split(label_df, test_size=0.2, random_state=42, stratify=label_df["label"])
    train_df_list.append(train_label)
    valid_df_list.append(valid_label)

train = pd.concat(train_df_list).drop(["label"], axis=1).reset_index(drop=True)
valid = pd.concat(valid_df_list).drop(["label"], axis=1).reset_index(drop=True)

# Save DataFrames to CSV and JSONL formats
test.to_csv("test.csv", index=False)
train.to_csv("train.csv", index=False)
valid.to_csv("valid.csv", index=False)

train.to_json("train.jsonl", orient='records', lines=True)
valid.to_json("valid.jsonl", orient='records', lines=True)
test.to_json("test.jsonl", orient='records', lines=True)

# Display the result
print("Test DataFrame:")
print(test)
print("\nTrain DataFrame:")
print(train)
print("\nValid DataFrame:")
print(valid)


Test DataFrame:
                                                prompt  \
0    Seit ca. 6 Jahren Shcmerzen re. Kniegelenk; Be...   
1    2019 erstmalig nach Ergometertrainiung Schmerz...   
2    Damals Judo und Kickboxen gemacht; kein größer...   
3    Vor 15 Jahren erstmalige Patellaluxation links...   
4    z.n Patellaluxation 1992; seit vielen Jahren K...   
..                                                 ...   
97   Ende 08/12 beim Fußall Distorsionstrauma ohne ...   
98   Z.n. LCA-Ersatz mit 4x ST 2017 bei Jürgen.  Da...   
99   Seit 25 Jahren Kampfsport; vor 12 Jahren Schme...   
100  Am 10.10.10 Kreuzbandriss beim Fußballspiel; P...   
101  seit Januar Schmerzen rechtes Knie ohne erinne...   

                                            completion  
0                        Aktivierte Gonarthrose rechts  
1        Krankheit der Patella; nicht näher bezeichnet  
2                              Chondromalacia patellae  
3                      Habituelle Luxation der Patella  
4 

In [ ]:
# %%bash
# python -m mlx_lm.convert --hf-path DiscoResearch/Llama3-DiscoLeo-Instruct-8B-v0.1 --mlx-path models_mlx/ -q 

[INFO] Loading


Fetching 10 files: 100%|██████████| 10/10 [00:00<00:00, 28093.13it/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


[INFO] Quantizing


In [ ]:
# %%bash
# mlx_lm.lora --help

usage: mlx_lm.lora [-h] [--model MODEL] [--train] [--data DATA]
                   [--lora-layers LORA_LAYERS] [--batch-size BATCH_SIZE]
                   [--iters ITERS] [--val-batches VAL_BATCHES]
                   [--learning-rate LEARNING_RATE]
                   [--steps-per-report STEPS_PER_REPORT]
                   [--steps-per-eval STEPS_PER_EVAL]
                   [--resume-adapter-file RESUME_ADAPTER_FILE]
                   [--adapter-path ADAPTER_PATH] [--save-every SAVE_EVERY]
                   [--test] [--test-batches TEST_BATCHES]
                   [--max-seq-length MAX_SEQ_LENGTH] [-c CONFIG]
                   [--grad-checkpoint] [--seed SEED] [--use-dora]

LoRA or QLoRA finetuning.

options:
  -h, --help            show this help message and exit
  --model MODEL         The path to the local model directory or Hugging Face
                        repo.
  --train               Do training
  --data DATA           Directory with {train, valid, test}.jsonl files
  -

## Training 

In [6]:

%%bash
python -m mlx_lm.lora --model models_mlx/ --data . --test --train --iters 1000

Loading pretrained model


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading datasets
Training
Trainable parameters: 0.021% (1.704M/8030.261M)
Starting training..., iters: 1000
Iter 1: Val loss 3.744, Val took 41.189s
Iter 10: Train loss 3.659, Learning Rate 1.000e-05, It/sec 0.348, Tokens/sec 322.616, Trained Tokens 9259, Peak mem 12.686 GB
Iter 20: Train loss 3.635, Learning Rate 1.000e-05, It/sec 0.381, Tokens/sec 328.917, Trained Tokens 17882, Peak mem 12.686 GB
Iter 30: Train loss 2.993, Learning Rate 1.000e-05, It/sec 0.314, Tokens/sec 325.324, Trained Tokens 28238, Peak mem 13.213 GB
Iter 40: Train loss 2.960, Learning Rate 1.000e-05, It/sec 0.358, Tokens/sec 338.686, Trained Tokens 37701, Peak mem 14.804 GB
Iter 50: Train loss 2.822, Learning Rate 1.000e-05, It/sec 0.373, Tokens/sec 330.531, Trained Tokens 46563, Peak mem 14.804 GB
Iter 60: Train loss 2.600, Learning Rate 1.000e-05, It/sec 0.303, Tokens/sec 309.994, Trained Tokens 56784, Peak mem 20.961 GB
Iter 70: Train loss 2.340, Learning Rate 1.000e-05, It/sec 0.291, Tokens/sec 266.466, Trai

In [3]:
import pandas as pd
df = pd.read_csv("test.csv")
df.head()

,prompt,label,completion
0,Seit ca. 6 Jahren Shcmerzen re. Kniegelenk; Be...,other,Aktivierte Gonarthrose rechts
1,2019 erstmalig nach Ergometertrainiung Schmerz...,other,Krankheit der Patella; nicht näher bezeichnet
2,Damals Judo und Kickboxen gemacht; kein größer...,other,Chondromalacia patellae
3,Vor 15 Jahren erstmalige Patellaluxation links...,other,Habituelle Luxation der Patella
4,z.n Patellaluxation 1992; seit vielen Jahren K...,other,funktionelles Patellaspitzensyndrom links


In [13]:
prompt = df["prompt"].iloc[2]
completion = df["completion"].iloc[2]

In [14]:
print(prompt)
print(completion)

Damals Judo und Kickboxen gemacht; kein größeres Trauma erinnerlich. Seit 2 Jahren immer wieder mal Schmerzen im rechtem Knie bei langer Belastung (ab 10 km Joggen oder 30 km Radfahren). Konservative Behandlung (6x Physiotherapie) ohne Erfolg. In letzter Zeit keine Schmerzprogredienz. Wünscht Zweitmeinung. Beinachse: 3QF varisch             Gangbild: flüssig re.Knie: keine Rötung; keine Überwärmung; deutliche Schwellung über med GS mit tastbarer Fluktation; 0/0/140; klinisch kein i.a. Erguss; Seitenbänder fest; Lachman neg. fest; Pivot shift neg.; hintere Schublade ++ unsicher; med step off deutlich vermindert; kein DS med GS; Steiman I pos; Payr pos. Instrumentelle Stabilitätsmessung:  re.   9  mm; li.9   mm Ap Translation in 90°:    re.  19  mm; li.10   mm 43.0 male
Chondromalacia patellae


## Inference test

In [15]:
%%bash
python -m mlx_lm.generate --model models_mlx \
                --max-tokens 120 \
               --adapter-path adapters \
               --prompt "2019 erstmalig nach Ergometertrainiung Schmerzen; KnochenDamals Judo und Kickboxen gemacht; kein größeres Trauma erinnerlich. Seit 2 Jahren immer wieder mal Schmerzen im rechtem Knie bei langer Belastung (ab 10 km Joggen oder 30 km Radfahren). Konservative Behandlung (6x Physiotherapie) ohne Erfolg. In letzter Zeit keine Schmerzprogredienz. Wünscht Zweitmeinung. Beinachse: 3QF varisch             Gangbild: flüssig re.Knie: keine Rötung; keine Überwärmung; deutliche Schwellung über med GS mit tastbarer Fluktation; 0/0/140; klinisch kein i.a. Erguss; Seitenbänder fest; Lachman neg. fest; Pivot shift neg.; hintere Schublade ++ unsicher; med step off deutlich vermindert; kein DS med GS; Steiman I pos; Payr pos. Instrumentelle Stabilitätsmessung:  re.   9  mm; li.9   mm Ap Translation in 90°:    re.  19  mm; li.10   mm 43.0 male"

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Prompt: <|begin_of_text|><|start_header_id|>user<|end_header_id|>

2019 erstmalig nach Ergometertrainiung Schmerzen; KnochenDamals Judo und Kickboxen gemacht; kein größeres Trauma erinnerlich. Seit 2 Jahren immer wieder mal Schmerzen im rechtem Knie bei langer Belastung (ab 10 km Joggen oder 30 km Radfahren). Konservative Behandlung (6x Physiotherapie) ohne Erfolg. In letzter Zeit keine Schmerzprogredienz. Wünscht Zweitmeinung. Beinachse: 3QF varisch             Gangbild: flüssig re.Knie: keine Rötung; keine Überwärmung; deutliche Schwellung über med GS mit tastbarer Fluktation; 0/0/140; klinisch kein i.a. Erguss; Seitenbänder fest; Lachman neg. fest; Pivot shift neg.; hintere Schublade ++ unsicher; med step off deutlich vermindert; kein DS med GS; Steiman I pos; Payr pos. Instrumentelle Stabilitätsmessung:  re.   9  mm; li.9   mm Ap Translation in 90°:    re.  19  mm; li.10   mm 43.0 male<|eot_id|><|start_header_id|>assistant<|end_header_id|>


Sonstige Meniskusschädigungen: Hinterhor

## Test data validation

In [20]:
%%bash
python -m mlx_lm.lora \
    --model models_mlx \
    --adapter-path adapters \
    --data . \
    --test

Loading pretrained model


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading datasets
Testing
Test loss 1.859, Test ppl 6.414.
